In [ ]:
import tensorflow as tf
import keras
tf.enable_eager_execution ()
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from tensorflow.keras import layers
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
from IPython.display import clear_output
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession



In [ ]:
spark

In [ ]:
def load_data():
#Load data from Hive tables, see feature extraction ETL for more details#
    START_DATEINT = 20200505
    REGION = 'APAC'

    load_data_query = '''
        SELECT cast(profile_id as string) as profile_id,
            cast(account_id as string) as account_id,
            region_rollup_desc,
            signup_country_iso_code, 
            sub_tenure_days,
            account_total_sec_7d,
            account_total_sec_14d,
            account_total_sec_28d,
            account_days_with_play_7d,
            account_days_with_play_14d,
            account_days_with_play_28d,
            profile_total_sec_7d,
            profile_total_sec_14d,
            profile_total_sec_28d,
            profile_days_with_play_7d,
            profile_days_with_play_14d,
            profile_days_with_play_28d,
            profile_days_with_new_originals_play_7d,
            profile_days_with_new_originals_play_14d,
            profile_days_with_new_originals_play_28d,
            est_tenure_12m
        from dse.figment_profile_algo_features 
        where region_dateint = %i
        and region_rollup_desc=  \'%s\'
        limit 100000
        '''

    
    load_data_query = (load_data_query) % (START_DATEINT,
                                           REGION)
    features_data = spark.sql(load_data_query)

    #Convert the dataframe to Pandas
    all_data = features_data.select("*").toPandas()
    
#   print(all_data.head(10))
    return all_data

In [ ]:
load_data();

In [ ]:
#Prepare data for the model
#Convert all boolean columns to int
def bool_to_int(dataset, column_arr):
    dataset[column_arr] = dataset[column_arr].astype('int')
    return dataset

#one-hot encoder categorical features
def one_hot_encoder(dataset, feature):
    featureBinarizer = LabelBinarizer().fit(dataset[feature])
    feature_encoded = featureBinarizer.transform(dataset[feature])
    return feature_encoded

#Embedding for show, season and title ids
def feature_embed(dataset, feature):
    embedding_layer = layers.Embedding(900000081342889666, 100, input_length=32)
    feature_embedding = embedding_layer(tf.constant(dataset[feature]))
    return feature_embedding



def normalize_data(dataset):
    # create scaler
    scaler = MinMaxScaler()
    # fit scaler on data
    scaler.fit(dataset)
    # apply transform
    normalized = scaler.transform(dataset)
    return normalized


In [ ]:
def analyze_correlation(dataset):
    #Analyze the data correlation
    C_mat = dataset.corr()
    fig = plt.figure(figsize = (15,15))

    sb.heatmap(C_mat, vmax = .8, square = True)
    plt.savefig('./feature_correlation.png')
    plt.show()

In [ ]:
all_data = load_data()
    
    #if plot:
    # plot the correlation between features
analyze_correlation(all_data)


In [ ]:
def define_NN_model(data):
    #turn off the eager execution in tensorflow
    INPUT_LAYER = 25
    HIDDEN_LAYER1 = 20
    HIDDEN_LAYER2 = 20
    HIDDEN_LAYER3 = 20
    OUTPUT_LAYER = 1
    
    ACIVATION_FUNC = 'relu'
    OPTIMIZER = 'adam'
    METRIC = 'mean_absolute_error'
    
    tf.compat.v1.disable_eager_execution()

    NN_model = Sequential()

    # The Input Layer :
    NN_model.add(Dense(INPUT_LAYER, kernel_initializer='normal',input_dim = data.shape[1], activation=ACIVATION_FUNC))

    # The Hidden Layers :
    NN_model.add(Dense(HIDDEN_LAYER1, kernel_initializer='normal',activation=ACIVATION_FUNC))
    NN_model.add(Dense(HIDDEN_LAYER2, kernel_initializer='normal',activation=ACIVATION_FUNC))
    NN_model.add(Dense(HIDDEN_LAYER3, kernel_initializer='normal',activation=ACIVATION_FUNC))
 #   NN_model.add(Dense(HIDDEN_LAYER4, kernel_initializer='normal',activation=ACIVATION_FUNC))


    # The Output Layer :
    NN_model.add(Dense(OUTPUT_LAYER, kernel_initializer='normal',activation=ACIVATION_FUNC))

    # Compile the network :
    NN_model.compile(loss=METRIC, optimizer=OPTIMIZER, metrics=[METRIC])
    NN_model.summary()
    return NN_model

In [ ]:
def plot_predictions(orig_data, pred_data):
    #plot predictions and expected y labels
    p = pred_data[:10000]
    y = orig_data[:10000]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    orig, = ax.plot(range(y.shape[0]),y,'bs')
    pred, = ax.plot(range(y.shape[0]),p, 'g^')
    leg2 = ax.legend([orig,pred],['original data','prediction data'], loc='best')
    plt.ylabel('28 Day Profile Total Playtime')
    plt.xlabel('Netflix Profile')
    plt.title('Playtime Predictions')
    plt.savefig('./prediction_accuracy.png')
    plt.show()

In [ ]:
def main(plot=True):
    # Seed the randomness of the simulation so this outputs the same thing each time
    np.random.seed(0)
    
    EPOCHS = 40
    BATCH_SIZE = 1000
    VALIDATIION_SPLIT = 0.1
    
    #load all data for modeling
    all_data = load_data()
    
    #if plot:
    # plot the correlation between features
    analyze_correlation(all_data)
    #The prediction for profile_total_sec_28d, consider log for normal distribution
    target = np.log(all_data.profile_total_sec_28d.replace(0,1))
    #Prepare data for the model
    #Convert all boolean columns to int
#     bool_cols = ['is_season_finale', 'is_kids', 'is_original']
#     bool_to_int(all_data, bool_cols)
    
    #Identify continuous features
    continuous = ['sub_tenure_days',
            'account_total_sec_7d',
            'account_total_sec_14d',
            'account_total_sec_28d',
            'account_days_with_play_7d',
            'account_days_with_play_14d',
            'account_days_with_play_28d',
            'profile_total_sec_7d',
            'profile_total_sec_14d',
            'profile_total_sec_28d',
            'profile_days_with_play_7d',
            'profile_days_with_play_14d',
            'profile_days_with_play_28d',
            'profile_days_with_new_originals_play_7d',
            'profile_days_with_new_originals_play_14d',
            'profile_days_with_new_originals_play_28d',
            'est_tenure_12m']
    

    dataContinuous = all_data[continuous]
    
   # one-hot encode the language categorical data 
    region_onehot = one_hot_encoder(all_data,'region_rollup_desc')
    # one-hot encode the country categorical data 
    cntry_onehot = one_hot_encoder(all_data,'signup_country_iso_code')
    
    #Embedding for show, season and title ids
    profileId_emb = one_hot_encoder(all_data, 'profile_id')
    accountId_emb = one_hot_encoder(all_data, 'account_id')
    
    # Combining all of above transformed data
    dataX = np.hstack([dataContinuous, region_onehot, cntry_onehot, profileId_emb, accountId_emb])
    print('here')
  
    #Normalize data
    normalized_features = normalize_data(dataX)
    
    # Split into train and test dataset for 70:30
    train_X, test_X, train_y, test_y = train_test_split(normalized_features, target, test_size = 0.30, random_state = 10)
    
    # Create checkpoint for model training
    checkpoint_name = 'Weights.hdf5' 
    checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
    callbacks_list = [checkpoint]
    
    NN_model = define_NN_model(train_X)
    
    NN_model.summary()
    
    #Train the model NN_model

    NN_model.fit(train_X, train_y, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split = VALIDATIION_SPLIT,
             callbacks=[plot_losses])

    #Make predictions on test data
    predictions = NN_model.predict(test_X)
    print(predictions.shape)
    #if plot:
    # plot the correlation between features
    plot_predictions(test_y, predictions)

    return predictions

In [ ]:
class TrainingPlot(keras.callbacks.Callback):
    
    # This function is called when the training begins
    def on_train_begin(self, logs={}):
        # Initialize the lists for holding the logs, losses and accuracies
        self.losses = []
        self.accuracy = []
        self.val_losses = []
        self.val_accuracy = []
        self.logs = []
    
    # This function is called at the end of each epoch
    def on_epoch_end(self, epoch, logs={}):
        
        # Append the logs, losses and accuracies to the lists
        self.logs.append(logs)
        print(self.logs)
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_accuracy.append(logs.get('val_accuracy'))
        
        # Before plotting ensure at least 2 epochs have passed
        if len(self.losses) > 1:
            
            # Clear the previous plot
            clear_output(wait=True)
            N = np.arange(0, len(self.losses))
            
            # You can chose the style of your preference
            print(self.val_losses)
            print(self.losses)
            # print(plt.style.available) to see the available options
            plt.style.use("seaborn")
            
            # Plot train loss, train acc, val loss and val acc against epochs passed
            #plt.figure()
            plt.figure(figsize=(15,5))
            plt.plot(N, self.losses, label = "train_loss")
            #plt.plot(N, self.accuracy, label = "train_acc")
            plt.plot(N, self.val_losses, label = "val_loss")
            #plt.plot(N, self.val_accuracy, label = "val_acc")
            plt.title("Training Loss and Accuracy [Epoch {}]".format(epoch))
            plt.xlabel("Epoch #")
            plt.ylabel("Loss/Accuracy")
            plt.legend()
            plt.savefig('./NN_model_losses_plot.png')
            plt.show()

plot_losses = TrainingPlot()

In [ ]:
if __name__ == '__main__':
    main()